In [ ]:
# create_zone_enhanced.py

import cv2
import json
import numpy as np

class SmartZoneManager:
    """Mengelola zona merah dengan fitur auto-adjustment dan persistensi"""
    
    def __init__(self):
        self.zones = []
        self.zone_templates = {}
        self.video_metadata = {}
    
    def create_zone_interactive(self, video_path, output_json):
        """Membuat zona dengan interface yang lebih user-friendly"""
        points = []
        
        def draw_polygon(event, x, y, flags, param):
            if event == cv2.EVENT_LBUTTONDOWN:
                points.append((x, y))
                print(f"Point {len(points)}: ({x}, {y})")
            elif event == cv2.EVENT_RBUTTONDOWN and len(points) > 0:
                points.pop()
                print(f"Removed last point. Total points: {len(points)}")
        
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        if not ret:
            print("Gagal membuka video.")
            return False
        
        # Simpan metadata video
        height, width = frame.shape[:2]
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        clone = frame.copy()
        cv2.namedWindow("Set Zona Merah - ENHANCED", cv2.WINDOW_NORMAL)
        cv2.setMouseCallback("Set Zona Merah - ENHANCED", draw_polygon)
        
        print("=== INSTRUKSI ===")
        print("- LEFT CLICK: Tambah titik zona")
        print("- RIGHT CLICK: Hapus titik terakhir")
        print("- 's': Simpan zona")
        print("- 'r': Reset semua titik")
        print("- 'q': Keluar tanpa menyimpan")
        print("================")
        
        while True:
            temp = clone.copy()
            
            # Gambar grid helper untuk presisi
            for i in range(0, width, 50):
                cv2.line(temp, (i, 0), (i, height), (200, 200, 200), 1)
            for i in range(0, height, 50):
                cv2.line(temp, (0, i), (width, i), (200, 200, 200), 1)
            
            # Gambar titik dan garis zona
            for i, point in enumerate(points):
                cv2.circle(temp, point, 6, (0, 0, 255), -1)
                cv2.putText(temp, str(i+1), (point[0]+10, point[1]-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
                if i > 0:
                    cv2.line(temp, points[i-1], point, (0, 0, 255), 2)
            
            if len(points) > 2:
                cv2.line(temp, points[-1], points[0], (0, 0, 255), 2)
                cv2.fillPoly(temp, [np.array(points)], (0, 0, 255, 50))
            
            # Info panel
            cv2.rectangle(temp, (10, 10), (300, 100), (0, 0, 0), -1)
            cv2.putText(temp, f"Points: {len(points)}", (20, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            cv2.putText(temp, f"Video: {width}x{height}", (20, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            cv2.putText(temp, "s=Save r=Reset q=Quit", (20, 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            
            cv2.imshow("Set Zona Merah - ENHANCED", temp)
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('r'):
                points.clear()
                print("Reset all points")
            elif key == ord('s') and len(points) >= 3:
                # Simpan zona dengan metadata
                zone_data = {
                    'polygon': points,
                    'video_metadata': {
                        'width': width,
                        'height': height,
                        'fps': fps,
                        'total_frames': total_frames,
                        'video_path': video_path
                    },
                    'creation_timestamp': cv2.getTickCount()
                }
                
                with open(output_json, 'w') as f:
                    json.dump(zone_data, f, indent=2)
                print(f"Zona berhasil disimpan ke {output_json}")
                
                cap.release()
                cv2.destroyAllWindows()
                return True
        
        cap.release()
        cv2.destroyAllWindows()
        return False

def create_zone_interactive(video_path, output_json="zona_enhanced.json"):
    """Wrapper function untuk membuat zona"""
    manager = SmartZoneManager()
    return manager.create_zone_interactive(video_path, output_json)

# Example usage:
if __name__ == "__main__":
    # Ganti dengan path video Anda
    VIDEO_PATH = "test.mp4"
    OUTPUT_JSON = "zona_enhanced.json"
    
    print("Starting Enhanced Zone Creator...")
    success = create_zone_interactive(VIDEO_PATH, OUTPUT_JSON)
    
    if success:
        print(f"✅ Zona berhasil dibuat dan disimpan ke {OUTPUT_JSON}")
    else:
        print("❌ Gagal membuat zona")